In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('/Users/ting2liu/data/kiva/kiva_ds_csv/NEW/loans.csv', parse_dates=["POSTED_TIME", "PLANNED_EXPIRATION_TIME",
                               "DISBURSE_TIME", "RAISED_TIME"])

In [3]:
NEW_names = ["LOAN_ID","LOAN_NAME","ORIGINAL_LANGUAGE","DESCRIPTION","DESCRIPTION_TRANSLATED","FUNDED_AMOUNT","LOAN_AMOUNT","STATUS","IMAGE_ID","VIDEO_ID","ACTIVITY_NAME","SECTOR_NAME","LOAN_USE","COUNTRY_CODE","COUNTRY_NAME","TOWN_NAME","CURRENCY_POLICY","CURRENCY_EXCHANGE_COVERAGE_RATE","CURRENCY","PARTNER_ID","POSTED_TIME","PLANNED_EXPIRATION_TIME","DISBURSE_TIME","RAISED_TIME","LENDER_TERM","NUM_LENDERS_TOTAL","NUM_JOURNAL_ENTRIES","NUM_BULK_ENTRIES","TAGS","BORROWER_NAMES","BORROWER_GENDERS","BORROWER_PICTURED","REPAYMENT_INTERVAL","DISTRIBUTION_MODEL"]

In [4]:
OLD_names = ['id','name','original_language','original_description','translated_description','funded_amount','loan_amount','status','image_id','video_id','activity','sector','use','country_code','country_name','town','currency_policy', 'currency_exchange_coverage_rate','currency','partner_id','posted_time','planned_expiration_time','disbursed_time','funded_time','term_in_months','lender_count','journal_entries_count','bulk_journal_entries_count','tags','borrower_names','borrower_genders','borrower_pictured','repayment_interval','distribution_model']

In [5]:
dic = {}
for (k,v) in zip(NEW_names, OLD_names):
    dic[k]=v

In [6]:
df = df.rename(columns=dic)

In [7]:
df.head()

,id,name,original_language,original_description,translated_description,funded_amount,loan_amount,status,image_id,video_id,...,term_in_months,lender_count,journal_entries_count,bulk_journal_entries_count,tags,borrower_names,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,1511246.0,NaN,...,7.0,3,2,1,NaN,Aivy,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,1507947.0,NaN,...,8.0,11,2,1,NaN,Idalia Marizza,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,1512245.0,NaN,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",Aasia,female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,1514419.0,NaN,...,14.0,21,2,1,user_favorite,Gulmira,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,1506830.0,NaN,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",Ricky\t,male,true,bullet,field_partner


In [8]:
df['year'] = df['posted_time'].dt.year #create year column

In [9]:
df.shape

(1419607, 35)

In [10]:
df = df[(df['status'] == 'funded') | (df['status'] == 'expired')]  #only take rows with 'funded' or 'expired' status

In [11]:
df.shape

(1409421, 35)

In [12]:
df.dropna(subset=['borrower_genders'], inplace=True)  # not interested in rows where gender is missing
df.reset_index(drop=True,inplace=True)

In [13]:
df.shape

(1372748, 35)

In [15]:
data = df[['borrower_genders', 'loan_amount', 'sector', 'country_name', 'country_code', 'status', 'year']]  # cols

In [16]:
def genders_groups(x):
    x = x.split(', ')
    if len(x) == 1:
        return x[0]
    else:
        genders = Counter(x).keys()
        if 'female' in genders and 'male' not in genders:
            return 'female_group'
        elif 'female' in genders and 'male' in genders:
            return 'mixed_group'
        else:
            return 'male_group'

In [17]:
data['gender_group'] = data['borrower_genders'].map(genders_groups)

/Users/ting2liu/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
data.head()

,borrower_genders,loan_amount,sector,country_name,country_code,status,year,gender_group
0,female,125.0,Retail,Philippines,PH,funded,2014,female
1,female,400.0,Clothing,Honduras,HN,funded,2014,female
2,female,400.0,Retail,Pakistan,PK,funded,2014,female
3,female,625.0,Agriculture,Kyrgyzstan,KG,funded,2014,female
4,male,425.0,Agriculture,Philippines,PH,funded,2014,male


In [19]:
g = data.groupby(['country_name', 'country_code', 'year'])

In [20]:
asdf = g['loan_amount'].agg(['mean', 'size']).reset_index()
asdf

,country_name,country_code,year,mean,size
0,Afghanistan,AF,2007,493.035714,420
1,Afghanistan,AF,2008,986.554622,357
2,Afghanistan,AF,2009,859.853420,614
3,Afghanistan,AF,2010,832.560137,291
4,Afghanistan,AF,2014,6000.000000,1
5,Afghanistan,AF,2016,8000.000000,1
6,Albania,AL,2012,1242.131148,305
7,Albania,AL,2013,1404.583333,480
8,Albania,AL,2014,1503.146552,580
9,Albania,AL,2015,1323.293651,630


In [21]:
final_data = asdf.query('year != 2018 and year !=2006')

In [22]:
final_data

,country_name,country_code,year,mean,size
0,Afghanistan,AF,2007,493.035714,420
1,Afghanistan,AF,2008,986.554622,357
2,Afghanistan,AF,2009,859.853420,614
3,Afghanistan,AF,2010,832.560137,291
4,Afghanistan,AF,2014,6000.000000,1
5,Afghanistan,AF,2016,8000.000000,1
6,Albania,AL,2012,1242.131148,305
7,Albania,AL,2013,1404.583333,480
8,Albania,AL,2014,1503.146552,580
9,Albania,AL,2015,1323.293651,630


In [23]:
final_data.to_csv('loans_final_formap.csv', index=False)